# 1. Install Dependencies and Setup
Install necessary libraries for data collection and augmentation.

In [1]:
!pip install labelme tensorflow opencv-python matplotlib albumentations

# 2. Collect Images
Capture images from the webcam to build the dataset.

In [6]:
import os
import time
import uuid
import cv2

In [1]:
IMAGES_PATH = 'data\Images'
number_images = 80 # Number of images to collect

In [4]:
# Ensure directory exists
if not os.path.exists(IMAGES_PATH):
    os.makedirs(IMAGES_PATH)

cap = cv2.VideoCapture(0)
for imgnum in range(number_images):
    print('Collecting image {}'.format(imgnum))
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture frame")
        break

    imgname = os.path.join(IMAGES_PATH,f'{str(uuid.uuid1())}.jpg')
    cv2.imwrite(imgname, frame)
    cv2.imshow('frame', frame)
    time.sleep(0.5)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [5]:
# run labelme in terminal to label the images collected

# 3. Create Dataset
Load libraries for data processing.

In [2]:
import tensorflow as tf
import cv2
import json
import numpy as np
from matplotlib import pyplot as plt

In [3]:
# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    try:
        tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [4]:
def load_image(x): 
    byte_img = tf.io.read_file(x)
    img = tf.io.decode_jpeg(byte_img)
    return img

# 4. Partition Unaugmented Data

### 4.1 MANUALLY SPLT DATA INTO TRAIN TEST AND VAL

In [ ]:
import importlib, subprocess, sys
import splitfolders
import shutil

if os.path.exists('data/images'):
    # ensure single-class subdirectory
    dst_dir = os.path.join('data', 'images', 'all')
    os.makedirs(dst_dir, exist_ok=True)

    # move image files only (no labels/classes)
    for entry in os.listdir('data/images'):
        src_path = os.path.join('data/images', entry)
        if os.path.isfile(src_path) and entry.lower().endswith(('.jpg', '.jpeg', '.png')):
            dst_path = os.path.join(dst_dir, entry)
            if not os.path.exists(dst_path):
                shutil.move(src_path, dst_path)

    # splitfolders expects subdirectories under 'data/images' (e.g. 'all')
    splitfolders.ratio('data/images', output="data", seed=1337, ratio=(.7, .15, .15))

Copying files: 160 files [00:00, 904.22 files/s]


In [11]:
160*.7 #  140 to train

112.0

In [ ]:
160*.15 #  24 to val

24.0

In [13]:
160*.15 #  24 to test

24.0

# 4. Move Matching Labels AFTER Annotation with Labelme
Move the JSON labels created by LabelMe to the corresponding folders (train/test/val) if they match the images there.

In [21]:
for folder in ['train','test','val']:
    images_path = os.path.join('data', folder, 'images')
    if not os.path.exists(images_path):
        print(f"Warning: {images_path} does not exist. Skipping.")
        continue
        
    for file in os.listdir(images_path):
        
        filename = file.split('.')[0]+'.json'
        existing_filepath = os.path.join('data','labels', filename)
        if os.path.exists(existing_filepath): 
            if not os.path.exists(os.path.join('data', folder, 'labels')):
                os.makedirs(os.path.join('data', folder, 'labels'))
            
            new_filepath = os.path.join('data',folder,'labels',filename)
            os.replace(existing_filepath, new_filepath)  

# 5. Image Augmentation
Use Albumentations to augment the dataset (crop, flip, brightness, etc.)

In [22]:
import albumentations as alb

In [23]:
augmentor = alb.Compose([alb.RandomCrop(width=450, height=450), 
                         alb.HorizontalFlip(p=0.5), 
                         alb.RandomBrightnessContrast(p=0.2),
                         alb.RandomGamma(p=0.2), 
                         alb.RGBShift(p=0.2), 
                         alb.VerticalFlip(p=0.5)], 
                        keypoint_params=alb.KeypointParams(format='xy', label_fields=['class_labels']))

In [24]:
# Iterate through partitions and augment images
for partition in ['train', 'test', 'val']: 
    partition_path = os.path.join('data', partition, 'images')
    if not os.path.exists(partition_path):
        continue
        
    for image in os.listdir(partition_path):
        img = cv2.imread(os.path.join('data', partition, 'images', image))

        classes = [0,0]
        coords = [0,0,0.00001,0.00001]
        label_path = os.path.join('data', partition, 'labels', f'{image.split(".")[0]}.json')
        
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                label = json.load(f)
    
            if label['shapes'][0]['label']=='LeftEye': 
                classes[0] = 1
                coords[0] = np.squeeze(label['shapes'][0]['points'])[0]
                coords[1] = np.squeeze(label['shapes'][0]['points'])[1]

            if label['shapes'][0]['label']=='RightEye':
                classes[1] = 1
                coords[2] = np.squeeze(label['shapes'][0]['points'])[0]
                coords[3] = np.squeeze(label['shapes'][0]['points'])[1]

            if len(label['shapes']) > 1:     
                if label['shapes'][1]['label'] =='LeftEye': 
                    classes[0] = 1 
                    coords[0] = np.squeeze(label['shapes'][1]['points'])[0]
                    coords[1] = np.squeeze(label['shapes'][1]['points'])[1]

                if label['shapes'][1]['label'] =='RightEye': 
                    classes[1] = 1
                    coords[2] = np.squeeze(label['shapes'][1]['points'])[0]
                    coords[3] = np.squeeze(label['shapes'][1]['points'])[1]
            
            # Normalize coordinates by original image size (640x480)
            np.divide(coords, [640,480,640,480])
                
        try: 
            for x in range(120): # Create 120 versions of each image
                keypoints = [(coords[:2]), (coords[2:])]
                augmented = augmentor(image=img, keypoints=keypoints, class_labels=['LeftEye','RightEye'])
                
                # Ensure output directories exist
                if not os.path.exists(os.path.join('aug_data', partition, 'images')):
                    os.makedirs(os.path.join('aug_data', partition, 'images'))
                if not os.path.exists(os.path.join('aug_data', partition, 'labels')):
                    os.makedirs(os.path.join('aug_data', partition, 'labels'))

                cv2.imwrite(os.path.join('aug_data', partition, 'images', f'{image.split(".")[0]}.{x}.jpg'), augmented['image'])

                annotation = {}
                annotation['image'] = image
                annotation['class'] = [0,0]
                annotation['keypoints'] = [0,0,0,0]

                if os.path.exists(label_path):
                    if len(augmented['keypoints']) > 0: 
                        for idx, cl in enumerate(augmented['class_labels']):
                            if cl == 'LeftEye': 
                                annotation['class'][0] = 1 
                                annotation['keypoints'][0] = augmented['keypoints'][idx][0]
                                annotation['keypoints'][1] = augmented['keypoints'][idx][1]
                            if cl == 'RightEye': 
                                annotation['class'][1] = 1 
                                annotation['keypoints'][2] = augmented['keypoints'][idx][0]
                                annotation['keypoints'][3] = augmented['keypoints'][idx][1]
                                
                annotation['keypoints'] = list(np.divide(annotation['keypoints'], [450,450,450,450]))


                with open(os.path.join('aug_data', partition, 'labels', f'{image.split(".")[0]}.{x}.json'), 'w') as f:
                    json.dump(annotation, f)

        except Exception as e:
            print(e)